In [1]:
import twitter

#---------------------------------------------
# Define a Function to Login Twitter API


def oauth_login():
   # These numbers are linked to my account and allow me to utilize the twitter API
    CONSUMER_KEY = 'gCWdCixlcrIFTyrYFej64p88r'
    CONSUMER_SECRET = 'FdtNHEJak2dV8L3EaS6JniQGnsECkf8YshGe8iLwXxrxAjH1BF'
    BEARER = "AAAAAAAAAAAAAAAAAAAAAKBjOQEAAAAAnqWaxWZsf53T2zfohDuKCqerlPM%3D5iaVyEYBDn3ypumzv7f62R4Q9hRMAyV554wG1dhhRvAXDPsB2O"
    OAUTH_TOKEN = '1268941562-dlyypvsZhZxKngcu40SltOxORBDPcEPxzUJjmbO'
    OAUTH_TOKEN_SECRET = 'rFPzlfuOFVpINtV6BjwIrfAP1J5ip5pVqw3ZXqoDIXqNT'

    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)

    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api


#The below function loads in the data from twitter. q is the query term, count represents how many tweets to grab at once. 
def getData(q, count):
    search_results = twitter_api.search.tweets(q=q, count=count, lang='en')

    statuses = search_results['statuses']
    print(len(statuses))

# Iterate through for a total of 10 batches of results by following the cursor

    for _ in range(10):
        print("Length of statuses", len(statuses))
        try:
            next_results = search_results['search_metadata']['next_results']
            print(next_results)
        except KeyError:  # No more results when next_results doesn't exist
            break

    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([kv.split('=') for kv in next_results[1:].split("&")])
        print(kwargs)

        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']

    return statuses


In [2]:
twitter_api = oauth_login()


In [17]:
def getTimeline():
    res = twitter_api.statuses.user_timeline(screen_name = "glossier")
    return res

In [18]:
timeline = getTimeline()

In [22]:
timeline

[{'created_at': 'Thu Dec 02 15:01:47 +0000 2021',
  'id': 1466422349591416833,
  'id_str': '1466422349591416833',
  'text': "@Spookymelon_ DM us and we're here to help!",
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'Spookymelon_',
     'name': 'spooky melon',
     'id': 932507942569566209,
     'id_str': '932507942569566209',
     'indices': [0, 13]}],
   'urls': []},
  'source': '<a href="http://www.falcon.io" rel="nofollow">Falcon Social Media Management </a>',
  'in_reply_to_status_id': 1466421032999342100,
  'in_reply_to_status_id_str': '1466421032999342100',
  'in_reply_to_user_id': 932507942569566209,
  'in_reply_to_user_id_str': '932507942569566209',
  'in_reply_to_screen_name': 'Spookymelon_',
  'user': {'id': 2610470250,
   'id_str': '2610470250',
   'name': 'Glossier',
   'screen_name': 'glossier',
   'location': 'New York, NY',
   'description': 'Glossier Inc. is a people-powered beauty ecosystem ✨ Skin first, m

In [3]:
data = getData("Glossier", 100)

100
Length of statuses 100
?max_id=1467560746745835528&q=Glossier&lang=en&count=100&include_entities=1
{'max_id': '1467560746745835528', 'q': 'Glossier', 'lang': 'en', 'count': '100', 'include_entities': '1'}
Length of statuses 200
?max_id=1467265497381048319&q=Glossier&lang=en&count=100&include_entities=1
{'max_id': '1467265497381048319', 'q': 'Glossier', 'lang': 'en', 'count': '100', 'include_entities': '1'}
Length of statuses 300
?max_id=1466945540554321920&q=Glossier&lang=en&count=100&include_entities=1
{'max_id': '1466945540554321920', 'q': 'Glossier', 'lang': 'en', 'count': '100', 'include_entities': '1'}
Length of statuses 400
?max_id=1466727579461865472&q=Glossier&lang=en&count=100&include_entities=1
{'max_id': '1466727579461865472', 'q': 'Glossier', 'lang': 'en', 'count': '100', 'include_entities': '1'}
Length of statuses 500
?max_id=1466389349625442314&q=Glossier&lang=en&count=100&include_entities=1
{'max_id': '1466389349625442314', 'q': 'Glossier', 'lang': 'en', 'count': '10

In [2]:
import json
from textblob import TextBlob

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import html
import re
import pandas as pd
import datetime


In [5]:
#Saving data for later so its consistant
data = json.dumps(data)

file = open('moredata.txt', 'w')
file.write(data)
file.close


<function TextIOWrapper.close()>

In [12]:
#Opening saved data
file = open("moredata.txt", 'r')
data = file.read()
d = json.loads(data)

file2 = open("newdata.txt", 'r')
data2 = file2.read()
d2 = json.loads(data2)

file3 = open("data.txt", 'r')
data3 = file3.read()
d3 = json.loads(data3)


In [13]:
df = pd.DataFrame(d)
df2 = pd.DataFrame(d2)
df3 = pd.DataFrame(d3)


In [20]:
combo = df.append(df2).append(df3).reset_index()

In [22]:
export = combo[["created_at", "text", "entities", "retweet_count", "favorite_count"]]

In [27]:
df = df[~df.text.str.contains("@metacritic")]
d = json.loads(df.to_json(orient="records"))

In [23]:
export.to_csv("dataexport.csv")

## Running after collecting data starts here

In [3]:
allDataImport = pd.read_csv("dataexport.csv")

In [4]:
glossOnly = allDataImport[allDataImport["keep?"] == 1]

In [5]:
glossOnly = glossOnly.drop(["keep?", "Unnamed: 0"],axis=1)

In [6]:
noRT = glossOnly[~glossOnly["text"].str.contains("RT")]

In [7]:
noRT["date"] = noRT.created_at.apply(lambda x:datetime.datetime.strptime(x,'%a %b %d %H:%M:%S %z %Y').date())

C:\Users\ragop\AppData\Local\Temp/ipykernel_8108/623958655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noRT["date"] = noRT.created_at.apply(lambda x:datetime.datetime.strptime(x,'%a %b %d %H:%M:%S %z %Y').date())


In [8]:
tweets = noRT.reset_index(drop = True)
for i in range (len(tweets)):
    x = tweets["text"][i].replace( "\n" ," ") 
    #cleaning newline “\n” from the tweets
    tweets["text"][i] = html.unescape(x)
tweets['text'] = tweets["text"].str.lower()
tweets.head()

C:\Users\ragop\AppData\Local\Temp/ipykernel_8108/702788600.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["text"][i] = html.unescape(x)


,created_at,text,entities,retweet_count,favorite_count,date
0,Mon Dec 06 15:53:17 +0000 2021,omg my glossier order came with twooooooo big ...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06
1,Mon Dec 06 15:15:58 +0000 2021,omg theyre opening a @glossier store in london...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,1,2021-12-06
2,Mon Dec 06 15:04:02 +0000 2021,the amount of glossier products i own you’d th...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06
3,Mon Dec 06 14:53:59 +0000 2021,not someone in chat saying syks mom does his e...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,16,2021-12-06
4,Mon Dec 06 14:24:07 +0000 2021,@knoxdiver i really like glossier’s https://t....,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06


In [9]:
for i in range (len(tweets)):
    tweets["text"][i] = re.sub(r"(@[A-Za-z0–9_]+)|[^\w\s]|#|http\S+", "",
     tweets["text"][i])
tweets.head()

C:\Users\ragop\AppData\Local\Temp/ipykernel_8108/2442837603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["text"][i] = re.sub(r"(@[A-Za-z0–9_]+)|[^\w\s]|#|http\S+", "",


,created_at,text,entities,retweet_count,favorite_count,date
0,Mon Dec 06 15:53:17 +0000 2021,omg my glossier order came with twooooooo big ...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06
1,Mon Dec 06 15:15:58 +0000 2021,omg theyre opening a store in london this month,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,1,2021-12-06
2,Mon Dec 06 15:04:02 +0000 2021,the amount of glossier products i own youd thi...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06
3,Mon Dec 06 14:53:59 +0000 2021,not someone in chat saying syks mom does his e...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,16,2021-12-06
4,Mon Dec 06 14:24:07 +0000 2021,i really like glossiers,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06


In [10]:
sw = stopwords.words("english")
sw.remove("not")

In [11]:
listScores = []
sia = SentimentIntensityAnalyzer()
word_dict = {'omg':1}
sia.lexicon.update(word_dict)

for i in tweets["text"]:
    listScores.append((sia.polarity_scores(str(i)))['compound'])

tweets["sentiment"] = pd.Series(listScores)
tweets.head()


,created_at,text,entities,retweet_count,favorite_count,date,sentiment
0,Mon Dec 06 15:53:17 +0000 2021,omg my glossier order came with twooooooo big ...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.2500
1,Mon Dec 06 15:15:58 +0000 2021,omg theyre opening a store in london this month,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,1,2021-12-06,0.2500
2,Mon Dec 06 15:04:02 +0000 2021,the amount of glossier products i own youd thi...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.0000
3,Mon Dec 06 14:53:59 +0000 2021,not someone in chat saying syks mom does his e...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,16,2021-12-06,0.5423
4,Mon Dec 06 14:24:07 +0000 2021,i really like glossiers,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.4201


In [12]:
def sentiment_category(sentiment):
    label = ''
    if(sentiment > 0):
        label = 'positive'
    elif(sentiment == 0):
        label = 'neutral'
    else:
        label = 'negative'
    return(label)
    
tweets['sentiment_category'] = tweets['sentiment'].apply(sentiment_category)
tweets.head()

,created_at,text,entities,retweet_count,favorite_count,date,sentiment,sentiment_category
0,Mon Dec 06 15:53:17 +0000 2021,omg my glossier order came with twooooooo big ...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.2500,positive
1,Mon Dec 06 15:15:58 +0000 2021,omg theyre opening a store in london this month,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,1,2021-12-06,0.2500,positive
2,Mon Dec 06 15:04:02 +0000 2021,the amount of glossier products i own youd thi...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.0000,neutral
3,Mon Dec 06 14:53:59 +0000 2021,not someone in chat saying syks mom does his e...,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,16,2021-12-06,0.5423,positive
4,Mon Dec 06 14:24:07 +0000 2021,i really like glossiers,"{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.4201,positive


In [13]:
counts = tweets.sentiment_category.value_counts()

In [14]:
counts.values

array([183, 144,  66], dtype=int64)

In [15]:
import plotly.express as px
fig = px.pie(counts,values = counts.values, names = counts.index, title = "Proportion of Tweets with Each Sentiment")
fig.show()

In [16]:
means = tweets.groupby("sentiment_category").mean()

In [17]:
means

,retweet_count,favorite_count,sentiment
sentiment_category,,,
negative,0.030303,4.090909,-0.403962
neutral,0.680556,12.145833,0.000000
positive,0.601093,16.163934,0.493414


In [18]:
fig = px.bar(means, y='favorite_count',x=means.index, title = 'Mean Likes by Sentiment',
labels={'favorite_count':'Mean Likes','sentiment_category':'Sentiment'})

fig.show()

In [19]:
fig = px.bar(means, y='retweet_count', x=means.index, title='Mean Retweets by Sentiment',
 labels={'retweet_count': 'Mean Retweets', 'sentiment_category': 'Sentiment'})

fig.show()


In [20]:
neu = tweets[tweets['sentiment_category'] == 'neutral']
neg = tweets[tweets['sentiment_category']=='negative']
pos = tweets[tweets['sentiment_category']=='positive']

In [21]:
gruDate = tweets.groupby(['date','sentiment_category'],as_index=False).count()

In [22]:
fig = px.line(gruDate,x='date',y='text',color='sentiment_category',markers=True,labels={'text':'Count','date':'Date','sentiment_category':'Sentiment'},color_discrete_sequence=["#1b9e77",'#7570b3','#d95f02'],title='Tweets by Sentiment over Time')
fig.show()

In [23]:
tweets_to_token = tweets

for i in range(len(tweets_to_token)):
    tweets_to_token['text'][i] = word_tokenize(tweets_to_token["text"][i])

for i in range(len(tweets_to_token)):
    tweets_to_token['text'][i] = [word for word in tweets_to_token['text'][i] if not word in sw]

tweets_to_token.head()

C:\Users\ragop\AppData\Local\Temp/ipykernel_8108/2046347919.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ragop\AppData\Local\Temp/ipykernel_8108/2046347919.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,created_at,text,entities,retweet_count,favorite_count,date,sentiment,sentiment_category
0,Mon Dec 06 15:53:17 +0000 2021,"[omg, glossier, order, came, twooooooo, big, p...","{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.2500,positive
1,Mon Dec 06 15:15:58 +0000 2021,"[omg, theyre, opening, store, london, month]","{'hashtags': [], 'symbols': [], 'user_mentions...",0,1,2021-12-06,0.2500,positive
2,Mon Dec 06 15:04:02 +0000 2021,"[amount, glossier, products, youd, think, im, ...","{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.0000,neutral
3,Mon Dec 06 14:53:59 +0000 2021,"[not, someone, chat, saying, syks, mom, eyebro...","{'hashtags': [], 'symbols': [], 'user_mentions...",0,16,2021-12-06,0.5423,positive
4,Mon Dec 06 14:24:07 +0000 2021,"[really, like, glossiers]","{'hashtags': [], 'symbols': [], 'user_mentions...",0,0,2021-12-06,0.4201,positive


In [24]:
# Split raw data into posts and words for analysis.
posts = [p
         for p in neu.text]


words = [w.lower()
         for t in posts
         for w in t]


In [25]:
from collections import Counter

In [26]:
from prettytable import PrettyTable


pt = PrettyTable(field_names=["Word", 'Count'])
c = Counter(words)
[pt.add_row(kv) for kv in c.most_common()[:50]]
pt.align["Word"], pt.align['Count'] = 'l', 'r'  # Set column alignment



In [27]:
# Towards the bottom of this top ten we can see that it seems to be words that were part of popularly retweeted tweet. 
pt[0:10]

Word,Count
,1681
e,784
o,600
s,584
i,574
r,505
t,497
a,432
n,377
l,325


In [28]:
def getHashtags(entities):
    return entities['hashtags']

def isAd(tags):
    res = False
    for tag in tags:
        if tag["text"] == ("ad"):
            res = True
        if tag["text"] == ("adaffiliate"):
            res = True

    return res